In [63]:
import os
import numpy as np
import pandas as pd
import json

# Parameter Description

Subject and session

In [64]:
NIP = "jm100042"
session_label = "09" # "07" : Visual, "08" : Social, "09" : MathLang

Parameter description in an external json file

In [65]:
fileNameDescription = os.path.join(
  '../01-DescriptionFiles',
  NIP,
  'description.json'
)

description = dict()
with open(str(fileNameDescription), 'r') as f:
    description = json.load(f)

In [66]:
subId = description["subId"]
infos_participant = description['infos_participant']
acq_date = description["session"][session_label]['acq_date']
location = description["session"][session_label]['location']
fileIdAnat = description["session"][session_label]['fileIdAnat']
skipFiles = np.array(description["session"][session_label]['skipFiles'])

Default parameters
If the file Ids are wrong a skipFile Parameter exist in the description file

In [67]:
dico = {
    "07": {
        "fileIdFunc": {
           "rest": [14, 34],
            # "rest": [14, 34, 49],
            "retino": [18, 22, 26, 30],
            "catv": [45]
            },
        "runScript": {
            "rest": "runRestingState",
            "retino": "runRetino",
            "catv": "runVisualCategory"
            }
        },

    "08": {
        "fileIdFunc": {
            "rest": [22, 33],
            "emo": [14, 37],
            "tom": [18, 41]
            },
        "runScript": {
            "rest": "runRestingState",
            "emo": "runEmotion",
            "tom": "runToM"
            }
        },

    "09": {
        "fileIdFunc": {
            # "ml": [18, 22, 26, 37, 41]
            "ml": [14, 18, 22, 33, 37]
            # "rest": [41]
            },
        "runScript": {
            "ml": "runMathLanguage",
            "rest": "runRestingState"
            }
        }
    }


dictSession = dico[session_label]
fileIdFunc = dictSession["fileIdFunc"]
runScript = dictSession["runScript"]

Useful dir and prefix

In [68]:
bids_dir = os.path.join(
    '../03-BidsConversion',
    f'sub-{subId:02}',
    f'ses-{session_label}')

xpd_dir = os.path.join(
    "../02-StimFiles",
    NIP,
    f'session{session_label}')

In [69]:
prefix = f'sub-{subId:02}_ses-{session_label}'

# Prepare neurospin_to_bids

## Build tsv file

Correct file ids to account for skipped files

In [70]:
def c(fileId):
    
    fileId += np.sum(fileId >= skipFiles)
    # In case of successive skipFiles number and the fileId fall on the first one, it would be increased only by one ; it needs to be inccreased until it is not anymore a skipfile
    while fileId in skipFiles:
        fileId += 1
        
    return fileId

to_import string creation

In [71]:
to_import = f'({fileIdAnat}, "anat", "T1w"),'

for task, filesId in fileIdFunc.items():

    for run, fileId in enumerate(filesId):

        func = f'({c(fileId)}, "func", "task-{task}_dir-ap_run-{run+1:02}_bold"),'
        sbref = f'({c(fileId-1)}, "func", "task-{task}_dir-ap_run-{run+1:02}_sbref"),'
        fmap_pa = f'({c(fileId-3)}, "fmap", "acq-{task}_dir-pa_run-{run+1:02}_epi"),'

        to_import += func + sbref + fmap_pa

print('\n'.join(sorted(to_import[1:-2].split(sep='),('))))

to_import = '(' + to_import[:-1] + ')'

11, "fmap", "acq-ml_dir-pa_run-01_epi"
13, "func", "task-ml_dir-ap_run-01_sbref"
14, "func", "task-ml_dir-ap_run-01_bold"
15, "fmap", "acq-ml_dir-pa_run-02_epi"
17, "func", "task-ml_dir-ap_run-02_sbref"
18, "func", "task-ml_dir-ap_run-02_bold"
19, "fmap", "acq-ml_dir-pa_run-03_epi"
21, "func", "task-ml_dir-ap_run-03_sbref"
22, "func", "task-ml_dir-ap_run-03_bold"
26, "anat", "T1w"
30, "fmap", "acq-ml_dir-pa_run-04_epi"
32, "func", "task-ml_dir-ap_run-04_sbref"
33, "func", "task-ml_dir-ap_run-04_bold"
34, "fmap", "acq-ml_dir-pa_run-05_epi"
36, "func", "task-ml_dir-ap_run-05_sbref"
37, "func", "task-ml_dir-ap_run-05_bold"


tsv file

In [72]:
df = pd.DataFrame(index=[0], columns=["participant_id","NIP","infos_participant","session_label","acq_date","acq_label","location","to_import"] )
df["participant_id"] = f"sub-{subId:02}"
df["NIP"] = NIP
df["infos_participant"] = infos_participant
df["session_label"] = session_label
df["acq_date"] = acq_date
df["location"] = location
df["to_import"] = to_import
print(df)

df.to_csv("participants_to_import.tsv", sep='\t', index=False)

  participant_id       NIP infos_participant session_label    acq_date  \
0         sub-21  jm100042     {"sex" : "M"}            09  2023-11-23   

  acq_label location                                          to_import  
0       NaN       7t  ((26, "anat", "T1w"),(14, "func", "task-ml_dir...  


# Import data
This step should be done directly in a terminal

When on a neurospin station:

cd /neurospin/icortex/iCortexDatabase/06-fMRI/

neurospin_to_bids --dataset-name  03-BidsConversion/

In [11]:
# cd /neurospin/icortex/iCortexDatabase/06-fMRI/
# neurospin_to_bids --dataset-name  03-BidsConversion/

# fmap Management

## Extract fmap AP
Extract first volumes from the func file

In [73]:
for task, filesId in fileIdFunc.items():

    for run, fileId in enumerate(filesId):

        fileNameFmapAP = os.path.join(
            bids_dir,
            'fmap',
            f'{prefix}_acq-{task}_dir-ap_run-{run+1:02}_epi')

        fileNameFuncSbrefAP = os.path.join(
            bids_dir,
            'func',
            f'{prefix}_task-{task}_dir-ap_run-{run+1:02}_sbref')

        # Copy file
        os.system(f'cp {fileNameFuncSbrefAP}.nii.gz {fileNameFmapAP}.nii.gz')
        os.system(f'cp {fileNameFuncSbrefAP}.json {fileNameFmapAP}.json')

## Complete fmaps' json files

In [74]:
for task, filesId in fileIdFunc.items():

    for run, fileId in enumerate(filesId):

        # Declare IntendedFor files
        shortFileNameFunc = os.path.join(
            f'ses-{session_label}',
            'func',
            f'{prefix}_task-{task}_dir-ap_run-{run+1:02}_bold' )
        shortFileNameSbref = os.path.join(
            f'ses-{session_label}',
            'func',
            f'{prefix}_task-{task}_dir-ap_run-{run+1:02}_sbref' )

        # Update fmap AP json
        fileNameFmapAP = os.path.join(
            bids_dir,
            'fmap',
            f'{prefix}_acq-{task}_dir-ap_run-{run+1:02}_epi')

        descriptionFmapAP = dict()
        with open(f'{fileNameFmapAP}.json', 'r') as f:
            descriptionFmapAP = json.load(f)

        descriptionFmapAP['B0FieldIdentifier'] = f'pepolar_{task}{run+1:02}'
        descriptionFmapAP['IntendedFor'] = [
            f'{shortFileNameFunc}.nii.gz',
            f'{shortFileNameSbref}.nii.gz']

        with open(f'{fileNameFmapAP}.json', 'w') as f:
            json.dump(descriptionFmapAP,f, indent=2)

        # Update fmap PA json
        fileNameFmapPA = os.path.join(
            bids_dir,
            'fmap',
            f'{prefix}_acq-{task}_dir-pa_run-{run+1:02}_epi')
        
        descriptionFmapPA = dict()
        with open(f'{fileNameFmapPA}.json', 'r') as f:
            descriptionFmapPA = json.load(f)

        descriptionFmapPA['B0FieldIdentifier'] = f'pepolar_{task}{run+1:02}'
        descriptionFmapPA['IntendedFor'] = [
            f'{shortFileNameFunc}.nii.gz',
            f'{shortFileNameSbref}.nii.gz']

        with open(f'{fileNameFmapPA}.json', 'w') as f:
            json.dump(descriptionFmapPA,f, indent=2)

# Import event files

In [75]:
fileNamesXpd = os.listdir(xpd_dir)

In [76]:
fileIdTask = fileIdFunc.copy()
if "rest" in fileIdTask :
  fileIdTask.pop("rest")

for task, filesId in fileIdTask.items():

    for run, fileId in enumerate(filesId):
        
        print(fileNamesXpd)
        print(f"{runScript[task]}_{subId:02}")
        print(f"run{run+1:02}.xpd")
        
        if len(filesId) == 1:
            list = [fname for fname in fileNamesXpd if fname.startswith(f"{runScript[task]}_{subId:02}")]
        else :
            list = [fname for fname in fileNamesXpd if fname.startswith(f"{runScript[task]}_{subId:02}") and fname.endswith(f"run{run+1:02}.xpd")]
        
        if len(list) != 1 :
            print(f'Error : Found {len(list)} files instead of 1')
            
        xpdfile = os.path.join(xpd_dir, list[-1])



        events_df = pd.read_csv( xpdfile, header = 0, comment = '#' )

        # drop scanner triggers
        if task == 'emo' :
            events_df = events_df[ events_df.emotion != 't' ]
        if task == 'ml' :
            events_df = events_df[ events_df.sentence != 't' ]

        # convert time unit from ms to second
        events_df[ 'start_time' ] /= 1000
        events_df[ 'end_time' ] /= 1000
        

        # extract stimulus duration
        events_df['duration'] = events_df[ 'end_time' ] - events_df[ 'start_time']

        # rename columns to bids-compliant format
        events_df.rename( columns = { 'cond' : 'trial_type',
                                      'start_time':'onset'}, 
                          inplace=True )

        # relocate columns
        onset = events_df.pop('onset')
        duration = events_df.pop('duration')
        events_df.insert(0, 'onset', onset)
        events_df.insert(1, 'duration', duration)
        if task == 'emo' :
            events_df = events_df[ events_df.main_stimulus != 't' ]
            events_df = events_df[ events_df.emotion != 't' ]

        # Rename the file and makes copy
        eventFile = os.path.join( 
            bids_dir,
            'func',
            f'sub-{subId:02}_ses-{session_label}_task-{task}_dir-ap_run-{run+1:02}_events.tsv')

        events_df.to_csv( eventFile, sep = '\t', index = False )

['runMathLanguage_21_202311230900._run01.xpd', 'runMathLanguage_21_202311230925._run03.xpd', 'runMathLanguage_21_202311230947._run04.xpd', 'runMathLanguage_21_202311230959._run05.xpd', 'runMathLanguage_21_202311230913._run02.xpd']
runMathLanguage_21
run01.xpd
['runMathLanguage_21_202311230900._run01.xpd', 'runMathLanguage_21_202311230925._run03.xpd', 'runMathLanguage_21_202311230947._run04.xpd', 'runMathLanguage_21_202311230959._run05.xpd', 'runMathLanguage_21_202311230913._run02.xpd']
runMathLanguage_21
run02.xpd
['runMathLanguage_21_202311230900._run01.xpd', 'runMathLanguage_21_202311230925._run03.xpd', 'runMathLanguage_21_202311230947._run04.xpd', 'runMathLanguage_21_202311230959._run05.xpd', 'runMathLanguage_21_202311230913._run02.xpd']
runMathLanguage_21
run03.xpd
['runMathLanguage_21_202311230900._run01.xpd', 'runMathLanguage_21_202311230925._run03.xpd', 'runMathLanguage_21_202311230947._run04.xpd', 'runMathLanguage_21_202311230959._run05.xpd', 'runMathLanguage_21_202311230913._r

# Run fMRIprep
You can now run fMRIprep